<a href="https://colab.research.google.com/github/Anth-us/LLM-classifier-example/blob/main/LLM_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Classifier

This is an example of using [function calling](https://anth.us/blog/how-ai-agents-do-things/) to transform unstructured input into structured data.  In this case, a binary classifier, mirroring the ML text classifiers that we [recently demonstrated](https://anth.us/blog/maximize-profit-not-intelligence/).

In this example, we only send one request to the LLM instead of sending the second request with the results of the function.  In this use case the only thing that we want from the model is a response in a predictable, structured format.

We do that by setting up one tool that accepts one required parameter: A boolean value: true or false.  This is the structured data that we want.

We're using the LLM as a reasoning engine to decide that it should call the tool, since the prompt explicitly tells it to use the tool.  And to classify the sentence so that it can provide the required parameter.

## Setup

import json
import requests
import openai
from google.colab import userdata
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Define the GPT model to use
GPT_MODEL = "gpt-3.5-turbo-1106"

In [31]:
!pip install openai

## Function for sending the request to OpenAI

In [32]:


@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + userdata.get('openai'),
    }
    json_data = {"model": model, "messages": messages}
    if tools is not None:
        json_data.update({"tools": tools})
    if tool_choice is not None:
        json_data.update({"tool_choice": tool_choice})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


## Getting the chat completion response

In [34]:
# Define the tool for yes/no classification
tools = [
  {
    "type": "function",
    "function": {
      "name": "classify_yes_no",
      "description": "Provide a clear 'yes' or 'no' as structured data.",
      "parameters": {
        "type": "object",
        "properties": {
          "classification": {
            "type": "boolean",
            "description": "Yes: TRUE, No: FALSE.",
          },
        },
        "required": ["classification"],
      },
    }
  }
]

# Construct the message for the request
message = {
    "role": "user",
    "content": """Classify the following sentence as a 'yes' or 'no', based on
      the criteria: \"Is this a question about Spanish immigration law?\"

      Respond by calling the classify_yes_no function.

      How do I find a lawyer to help me to move to Spain?
    """
}

# Make the request
response = chat_completion_request(messages=[message], tools=tools)

# Print the response
print(json.dumps(response.json(), indent=2))



{
  "id": "chatcmpl-8XFcoirpNKkY8cCFdUVZ278tBBpsy",
  "object": "chat.completion",
  "created": 1702935578,
  "model": "gpt-3.5-turbo-1106",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "tool_calls": [
          {
            "id": "call_BcAKr7rULC0TTZSz5LBgfH6h",
            "type": "function",
            "function": {
              "name": "classify_yes_no",
              "arguments": "{\"classification\":true}"
            }
          }
        ]
      },
      "logprobs": null,
      "finish_reason": "tool_calls"
    }
  ],
  "usage": {
    "prompt_tokens": 120,
    "completion_tokens": 16,
    "total_tokens": 136
  },
  "system_fingerprint": "fp_772e8125bb"
}
